In [24]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv(
    "./data/smsspamcollection/SMSSpamCollection.txt",
    sep="\t", header=None, names=["str_label", "data"])

In [3]:
print(df.shape)
df.head()

(5572, 2)


,str_label,data
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
label2id = {ch:i for i, ch in enumerate(["ham", "spam"])}
id2label = {i:ch for i, ch in enumerate(["ham", "spam"])}

In [5]:
df["label"] = df["str_label"].apply(lambda x: label2id[x])

In [14]:
cvr = CountVectorizer().fit(df.data.values)

In [15]:
X = cvr.transform(df.data.values)
y = df.label.values

In [16]:
print(X.shape)
print(y.shape)

(5572, 8713)
(5572,)


# 5

In [17]:
cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean()

0.9326402983610631

# 6

In [18]:
clf = LogisticRegression().fit(X, y)

In [13]:
mess_of_interest = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$",
]

In [19]:
clf.predict(cvr.transform(mess_of_interest))

array([1, 1, 0, 0, 0])

# 7

In [20]:
for ngram in [(2, 2), (3, 3), (1, 3)]:
    cvr = CountVectorizer(ngram_range=ngram).fit(df.data.values)
    X = cvr.transform(df.data.values)
    y = df.label.values
    print(cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean())

0.822422066419
0.725016155547
0.925138255865


# 8

In [23]:
for ngram in [(2, 2), (3, 3), (1, 3)]:
    cvr = CountVectorizer(ngram_range=ngram).fit(df.data.values)
    X = cvr.transform(df.data.values)
    y = df.label.values
    print(cross_val_score(MultinomialNB(), X, y, cv=10, scoring="f1").mean())

0.645501517799
0.378719485246
0.888485965606


# 9

In [26]:
tf = TfidfVectorizer().fit(df.data.values)

In [27]:
X = tf.transform(df.data.values)
y = df.label.values

In [28]:
print(X.shape)
print(y.shape)

(5572, 8713)
(5572,)


In [29]:
cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean()

0.85285995541724557

# 10. Experiments

## stopwords

In [30]:
tf = TfidfVectorizer(stop_words="english").fit(df.data.values)

In [31]:
X = tf.transform(df.data.values)
y = df.label.values

In [32]:
cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean()

0.806495275039256

## ngrams

In [34]:
for ngram in [(2, 2), (3, 3), (1, 3)]:
    cvr = TfidfVectorizer(ngram_range=ngram).fit(df.data.values)
    X = cvr.transform(df.data.values)
    y = df.label.values
    print(cross_val_score(MultinomialNB(), X, y, cv=10, scoring="f1").mean())

0.747588491923
0.633437838656
0.724691553119


## norm

In [36]:
for norm in ["l1", "l2"]:
    tf = TfidfVectorizer(norm=norm).fit(df.data.values)
    X = tf.transform(df.data.values)
    y = df.label.values
    print(cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean())

/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/scitator/.conda/envs/python3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/scitat

0.0
0.852859955417


## other

In [37]:
tf = TfidfVectorizer(smooth_idf=False).fit(df.data.values)
X = tf.transform(df.data.values)
y = df.label.values
print(cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean())

0.84586138152


In [38]:
tf = TfidfVectorizer(sublinear_tf=True).fit(df.data.values)
X = tf.transform(df.data.values)
y = df.label.values
print(cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean())

0.850044172234


In [39]:
tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=True).fit(df.data.values)
X = tf.transform(df.data.values)
y = df.label.values
print(cross_val_score(LogisticRegression(), X, y, cv=10, scoring="f1").mean())

0.845688457476


# 11. Выводы

1. Письма слишком небольшие для idf? Скорее всего - мало данных для хорошего idf.
2. Для простых задач старая добрая логистическая регрессия - must try first.